In [16]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
import talib

# Initialize TVDatafeed (use your TradingView credentials if needed; here we use the free version)
tv = TvDatafeed()

# Function to fetch data from TradingView
def fetch_data(symbol, exchange, n_bars=500):
    """
    Fetch historical daily data for a given symbol and exchange.
    
    Args:
        symbol (str): Instrument symbol (e.g., 'AAPL')
        exchange (str): Exchange name (e.g., 'NASDAQ')
        n_bars (int): Number of bars to fetch (default: 5000)
    
    Returns:
        pd.DataFrame: Historical data
    """
    try:
        data = tv.get_hist(symbol=symbol, exchange=exchange, interval=Interval.in_daily, n_bars=n_bars)
        if data is None or data.empty:
            print(f"No data retrieved for {symbol} on {exchange}")
            return None
        return data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to calculate direction indicators and classify direction
def calculate_direction(df):
    """
    Calculate nine direction indicators and determine market direction via voting.
    
    Args:
        df (pd.DataFrame): DataFrame with OHLC data
    
    Returns:
        pd.DataFrame: DataFrame with direction classification
    """
    # Calculate indicators
    df['sma10'] = talib.SMA(df['close'], timeperiod=10)
    df['sma30'] = talib.SMA(df['close'], timeperiod=30)
    df['ema10'] = talib.EMA(df['close'], timeperiod=10)
    df['ema30'] = talib.EMA(df['close'], timeperiod=30)
    macd, signal, _ = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['macd'] = macd
    df['macd_signal'] = signal
    df['rsi'] = talib.RSI(df['close'], timeperiod=14)
    df['adx'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['plus_di'] = talib.PLUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
    df['minus_di'] = talib.MINUS_DI(df['high'], df['low'], df['close'], timeperiod=14)
    df['sar'] = talib.SAR(df['high'], df['low'])
    df['stoch_k'], df['stoch_d'] = talib.STOCH(df['high'], df['low'], df['close'], fastk_period=14, slowk_period=3, slowd_period=3)
    df['linreg_slope'] = talib.LINEARREG_SLOPE(df['close'], timeperiod=20)

    # Define direction votes for each indicator
    df['dir_sma'] = np.where(df['sma10'] > df['sma30'] * 1.005, 'up', 
                            np.where(df['sma10'] < df['sma30'] * 0.995, 'down', 'sideways'))
    df['dir_ema'] = np.where(df['ema10'] > df['ema30'] * 1.005, 'up', 
                            np.where(df['ema10'] < df['ema30'] * 0.995, 'down', 'sideways'))
    df['dir_macd'] = np.where(df['macd'] > df['macd_signal'], 'up', 
                             np.where(df['macd'] < df['macd_signal'], 'down', 'sideways'))
    df['dir_rsi'] = pd.cut(df['rsi'], bins=[0, 40, 60, 100], labels=['down', 'sideways', 'up'], right=False)
    df['dir_adx'] = np.where((df['adx'] > 25) & (df['plus_di'] > df['minus_di']), 'up',
                            np.where((df['adx'] > 25) & (df['plus_di'] < df['minus_di']), 'down',
                                    np.where(df['adx'] < 20, 'sideways', 'nan')))
    df['dir_sar'] = np.where(df['sar'] < df['close'], 'up', 'down')
    df['dir_stoch'] = np.where((df['stoch_k'] > df['stoch_d']) & (df['stoch_k'] > 50), 'up',
                              np.where((df['stoch_k'] < df['stoch_d']) & (df['stoch_k'] < 50), 'down', 'sideways'))
    df['dir_linreg'] = np.where(df['linreg_slope'] > 0, 'up', 
                               np.where(df['linreg_slope'] < 0, 'down', 'sideways'))

    # List of direction vote columns
    dir_cols = ['dir_sma', 'dir_ema', 'dir_macd', 'dir_rsi', 'dir_adx', 'dir_sar', 'dir_stoch', 'dir_linreg']

    # Determine final direction by plurality vote
    df['direction'] = df[dir_cols].mode(axis=1)[0]
    return df

# Function to calculate volatility indicators and classify volatility
def calculate_volatility(df):
    """
    Calculate nine volatility indicators and determine market volatility via voting.
    
    Args:
        df (pd.DataFrame): DataFrame with OHLC data
    
    Returns:
        pd.DataFrame: DataFrame with volatility classification
    """
    # Calculate volatility indicators
    df['atr14'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['atr50'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=50)
    df['std_close20'] = df['close'].rolling(20).std()
    df['std_close50'] = df['close'].rolling(50).std()
    upper, middle, lower = talib.BBANDS(df['close'], timeperiod=20, nbdevup=2, nbdevdn=2)
    df['bb_width'] = (upper - lower) / middle
    df['chaikin_vol'] = talib.ROC(talib.EMA(df['high'] - df['low'], timeperiod=10), timeperiod=10)
    df['hist_vol20'] = df['close'].pct_change().rolling(20).std() * np.sqrt(252)
    df['hist_vol50'] = df['close'].pct_change().rolling(50).std() * np.sqrt(252)
    df['avg_range20'] = (df['high'] - df['low']).rolling(20).mean()

    # List of volatility indicators (fixed line)
    vol_indicators = ['atr14', 'atr50', 'std_close20', 'std_close50', 'bb_width', 'chaikin_vol', 'hist_vol20', 'hist_vol50', 'avg_range20']

    # Classify each indicator based on historical percentiles (30th and 70th)
    for ind in vol_indicators:
        df[f'{ind}_p30'] = df[ind].rolling(252).quantile(0.3)
        df[f'{ind}_p70'] = df[ind].rolling(252).quantile(0.7)
        df[f'vol_{ind}'] = np.where(df[ind] > df[f'{ind}_p70'], 'high',
                                   np.where(df[ind] < df[f'{ind}_p30'], 'low', 'average'))

    # List of volatility vote columns
    vol_cols = [f'vol_{ind}' for ind in vol_indicators]

    # Determine final volatility by plurality vote
    df['volatility'] = df[vol_cols].mode(axis=1)[0]
    return df

# Function to determine market regime
def determine_regime(df):
    """
    Combine direction and volatility into a market regime.
    
    Args:
        df (pd.DataFrame): DataFrame with direction and volatility
    
    Returns:
        pd.DataFrame: DataFrame with regime column
    """
    df['regime'] = df['direction'] + '-' + df['volatility']
    return df

# Function to calculate 2-Period RSI signals
def calculate_rsi_signals(df):
    """
    Calculate 2-Period RSI signals based on Larry Connors' strategy.
    
    Args:
        df (pd.DataFrame): DataFrame with OHLC data
    
    Returns:
        pd.DataFrame: DataFrame with RSI signals
    """
    # Calculate 200-day SMA for trend filter
    df['sma200'] = talib.SMA(df['close'], timeperiod=200)
    # Calculate 5-day SMA for exit
    df['sma5'] = talib.SMA(df['close'], timeperiod=5)
    # Calculate 2-period RSI
    df['rsi2'] = talib.RSI(df['close'], timeperiod=2)

    # Generate buy/sell signals
    # Buy: Price > 200-day SMA and RSI2 < 10
    # Sell: Price < 200-day SMA and RSI2 > 90
    df['signal'] = np.where((df['close'] > df['sma200']) & (df['rsi2'] < 30), 1,  # Buy
                           np.where((df['close'] < df['sma200']) & (df['rsi2'] > 70), -1, 0))  # Sell

    return df

# Function to simulate trades based on 2-Period RSI strategy
def simulate_trades(df):
    """
    Implement Larry Connors' 2-Period RSI strategy and simulate trades, tracking regimes.
    
    Args:
        df (pd.DataFrame): DataFrame with OHLC, signal, and regime data
    
    Returns:
        pd.DataFrame: DataFrame of trades with regime and return
    """
    trades = []
    position = 0  # 0: no position, 1: long, -1: short
    entry_price = 0
    entry_regime = ''
    entry_date = None

    for i, row in df.iterrows():
        # Entry: Buy (1) or Sell (-1)
        if row['signal'] == 1 and position == 0:  # Buy signal
            position = 1
            entry_price = row['close']
            entry_regime = row['regime']
            entry_date = i
        elif row['signal'] == -1 and position == 0:  # Sell signal
            position = -1
            entry_price = row['close']
            entry_regime = row['regime']
            entry_date = i

        # Exit: Long exit when close > 5-day SMA, Short exit when close < 5-day SMA
        if position == 1 and row['close'] > row['sma5']:  # Exit long
            trade_return = (row['close'] - entry_price) / entry_price
            trades.append({'entry_date': entry_date, 'exit_date': i, 'regime': entry_regime, 'return': trade_return})
            position = 0
        elif position == -1 and row['close'] < row['sma5']:  # Exit short
            trade_return = (entry_price - row['close']) / entry_price  # Short return
            trades.append({'entry_date': entry_date, 'exit_date': i, 'regime': entry_regime, 'return': trade_return})
            position = 0

    return pd.DataFrame(trades)

# Function to calculate performance metrics
def calculate_metrics(trades_df):
    """
    Calculate performance metrics including win rate and average return.
    
    Args:
        trades_df (pd.DataFrame): DataFrame of trades
    
    Returns:
        dict: Metrics including overall and per-regime win rates and average returns
    """
    if trades_df.empty:
        return {
            'overall_win_rate': np.nan,
            'overall_avg_return': np.nan,
            'regime_win_rates': {},
            'regime_avg_returns': {},
            'regime_trade_counts': {}
        }

    # Overall metrics
    total_trades = len(trades_df)
    overall_win_rate = (trades_df['return'] > 0).mean()
    overall_avg_return = trades_df['return'].mean()

    # Per-regime metrics
    regime_win_rates = trades_df.groupby('regime').apply(lambda x: (x['return'] > 0).mean() if len(x) > 0 else np.nan).to_dict()
    regime_avg_returns = trades_df.groupby('regime')['return'].mean().to_dict()
    regime_trade_counts = trades_df['regime'].value_counts().to_dict()

    return {
        'overall_win_rate': overall_win_rate,
        'overall_avg_return': overall_avg_return,
        'total_trades': total_trades,
        'regime_win_rates': regime_win_rates,
        'regime_avg_returns': regime_avg_returns,
        'regime_trade_counts': regime_trade_counts
    }

# Main function to analyze an instrument
def analyze_instrument(symbol, exchange):
    """
    Analyze a single instrument: fetch data, define regimes, run 2-Period RSI strategy, and calculate metrics.
    
    Args:
        symbol (str): Instrument symbol
        exchange (str): Exchange name
    """
    # Fetch data
    df = fetch_data(symbol, exchange, n_bars=1000)
    if df is None:
        return

    print(f"Instrument: {symbol} ({exchange})")
    print(f"Initial data rows fetched: {len(df)} (approx. {len(df)/252:.2f} years)")

    # Calculate direction, volatility, and regimes
    df = calculate_direction(df)
    df = calculate_volatility(df)
    df = determine_regime(df)

    # Calculate 2-Period RSI signals
    df = calculate_rsi_signals(df)

    # Drop rows with insufficient data (e.g., NaNs from indicators or regimes)
    df = df.dropna(subset=['regime', 'sma200', 'sma5', 'rsi2', 'signal'])
    print(f"Data rows after dropping NaNs: {len(df)} (approx. {len(df)/252:.2f} years)")

    # Simulate trades
    trades_df = simulate_trades(df)
    print(f"Number of trades: {len(trades_df)}")

    # Calculate performance metrics
    metrics = calculate_metrics(trades_df)

    # Print results
    print(f"Overall Win Rate: {metrics['overall_win_rate']:.2%}")
    print(f"Overall Average Trade Return: {metrics['overall_avg_return']:.2%}")
    print(f"Total Trades: {metrics['total_trades']}")
    print("Regime Performance:")
    for regime in sorted(metrics['regime_win_rates'].keys()):
        win_rate = metrics['regime_win_rates'].get(regime, np.nan)
        avg_return = metrics['regime_avg_returns'].get(regime, np.nan)
        count = metrics['regime_trade_counts'].get(regime, 0)
        print(f"  {regime}: Win Rate = {win_rate:.2%}, Avg Return = {avg_return:.2%} ({count} trades)")
    print("\n")

# Example usage with multiple instruments
if __name__ == "__main__":
    # Define a list of instruments to analyze
    instruments = [
        {'symbol': 'EURUSD', 'exchange': 'OANDA'},
        {'symbol': 'NAS100USD', 'exchange': 'OANDA'},
        {'symbol': 'XAUUSD', 'exchange': 'OANDA'},
        {'symbol': 'ES1!', 'exchange': 'CME_MINI'},
        {'symbol': 'CORNUSD', 'exchange': 'OANDA'},
        {'symbol': 'WTICOUSD', 'exchange': 'OANDA'},
        # Add more instruments as needed, e.g., {'symbol': 'ES1!', 'exchange': 'CME'} for S&P 500 futures
    ]

    # Run analysis for each instrument
    for inst in instruments:
        analyze_instrument(inst['symbol'], inst['exchange'])

Instrument: EURUSD (OANDA)
Initial data rows fetched: 1000 (approx. 3.97 years)
Data rows after dropping NaNs: 801 (approx. 3.18 years)
Number of trades: 84
Overall Win Rate: 70.24%
Overall Average Trade Return: 0.07%
Total Trades: 84
Regime Performance:
  down-average: Win Rate = 77.78%, Avg Return = 0.24% (9 trades)
  down-high: Win Rate = 90.91%, Avg Return = 0.36% (11 trades)
  down-low: Win Rate = 68.75%, Avg Return = 0.14% (16 trades)
  sideways-average: Win Rate = 50.00%, Avg Return = -0.30% (6 trades)
  sideways-high: Win Rate = 33.33%, Avg Return = -1.28% (6 trades)
  sideways-low: Win Rate = 63.64%, Avg Return = 0.02% (11 trades)
  up-average: Win Rate = 100.00%, Avg Return = 0.61% (5 trades)
  up-high: Win Rate = 55.56%, Avg Return = 0.05% (9 trades)
  up-low: Win Rate = 81.82%, Avg Return = 0.31% (11 trades)




C:\Users\rs\AppData\Local\Temp\ipykernel_7740\2324371025.py:231: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regime_win_rates = trades_df.groupby('regime').apply(lambda x: (x['return'] > 0).mean() if len(x) > 0 else np.nan).to_dict()


Instrument: NAS100USD (OANDA)
Initial data rows fetched: 1000 (approx. 3.97 years)
Data rows after dropping NaNs: 801 (approx. 3.18 years)
Number of trades: 87
Overall Win Rate: 71.26%
Overall Average Trade Return: 0.57%
Total Trades: 87
Regime Performance:
  down-average: Win Rate = 75.00%, Avg Return = 1.37% (16 trades)
  down-high: Win Rate = 72.73%, Avg Return = 0.85% (11 trades)
  down-low: Win Rate = 80.00%, Avg Return = 0.69% (5 trades)
  sideways-average: Win Rate = 57.14%, Avg Return = -0.22% (7 trades)
  sideways-high: Win Rate = 100.00%, Avg Return = 1.45% (1 trades)
  sideways-low: Win Rate = 100.00%, Avg Return = 1.65% (1 trades)
  up-average: Win Rate = 58.82%, Avg Return = -0.30% (17 trades)
  up-high: Win Rate = 60.00%, Avg Return = 0.27% (5 trades)
  up-low: Win Rate = 79.17%, Avg Return = 0.72% (24 trades)




C:\Users\rs\AppData\Local\Temp\ipykernel_7740\2324371025.py:231: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regime_win_rates = trades_df.groupby('regime').apply(lambda x: (x['return'] > 0).mean() if len(x) > 0 else np.nan).to_dict()


Instrument: XAUUSD (OANDA)
Initial data rows fetched: 1000 (approx. 3.97 years)
Data rows after dropping NaNs: 801 (approx. 3.18 years)
Number of trades: 77
Overall Win Rate: 72.73%
Overall Average Trade Return: 0.12%
Total Trades: 77
Regime Performance:
  down-average: Win Rate = 66.67%, Avg Return = -0.00% (12 trades)
  down-high: Win Rate = 87.50%, Avg Return = 0.36% (8 trades)
  down-low: Win Rate = 42.86%, Avg Return = -1.37% (7 trades)
  sideways-average: Win Rate = 83.33%, Avg Return = 0.23% (6 trades)
  sideways-high: Win Rate = 75.00%, Avg Return = 0.68% (4 trades)
  sideways-low: Win Rate = 66.67%, Avg Return = 0.57% (3 trades)
  up-average: Win Rate = 66.67%, Avg Return = -0.26% (15 trades)
  up-high: Win Rate = 80.95%, Avg Return = 0.65% (21 trades)
  up-low: Win Rate = 100.00%, Avg Return = 0.52% (1 trades)




...

C:\Users\rs\AppData\Local\Temp\ipykernel_7740\2324371025.py:231: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regime_win_rates = trades_df.groupby('regime').apply(lambda x: (x['return'] > 0).mean() if len(x) > 0 else np.nan).to_dict()


Instrument: CORNUSD (OANDA)
Initial data rows fetched: 1000 (approx. 3.97 years)
Data rows after dropping NaNs: 801 (approx. 3.18 years)
Number of trades: 84
Overall Win Rate: 69.05%
Overall Average Trade Return: 0.04%
Total Trades: 84
Regime Performance:
  down-average: Win Rate = 75.00%, Avg Return = 0.28% (20 trades)
  down-high: Win Rate = 81.82%, Avg Return = 1.16% (11 trades)
  down-low: Win Rate = 53.85%, Avg Return = -0.10% (13 trades)
  sideways-average: Win Rate = 66.67%, Avg Return = -0.05% (3 trades)
  sideways-low: Win Rate = 75.00%, Avg Return = 0.27% (8 trades)
  up-average: Win Rate = 61.54%, Avg Return = -0.57% (13 trades)
  up-high: Win Rate = 80.00%, Avg Return = -2.30% (5 trades)
  up-low: Win Rate = 63.64%, Avg Return = 0.30% (11 trades)




C:\Users\rs\AppData\Local\Temp\ipykernel_7740\2324371025.py:231: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regime_win_rates = trades_df.groupby('regime').apply(lambda x: (x['return'] > 0).mean() if len(x) > 0 else np.nan).to_dict()


Instrument: WTICOUSD (OANDA)
Initial data rows fetched: 1000 (approx. 3.97 years)
Data rows after dropping NaNs: 801 (approx. 3.18 years)
Number of trades: 82
Overall Win Rate: 65.85%
Overall Average Trade Return: 0.26%
Total Trades: 82
Regime Performance:
  down-average: Win Rate = 70.37%, Avg Return = 0.41% (27 trades)
  down-high: Win Rate = 71.43%, Avg Return = 1.66% (7 trades)
  down-low: Win Rate = 57.89%, Avg Return = -0.61% (19 trades)
  sideways-average: Win Rate = 100.00%, Avg Return = 2.44% (1 trades)
  sideways-low: Win Rate = 50.00%, Avg Return = 0.07% (6 trades)
  up-average: Win Rate = 40.00%, Avg Return = -1.75% (5 trades)
  up-high: Win Rate = 100.00%, Avg Return = 4.89% (1 trades)
  up-low: Win Rate = 75.00%, Avg Return = 0.70% (16 trades)




C:\Users\rs\AppData\Local\Temp\ipykernel_7740\2324371025.py:231: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regime_win_rates = trades_df.groupby('regime').apply(lambda x: (x['return'] > 0).mean() if len(x) > 0 else np.nan).to_dict()
